In [1]:
import nltk
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import nltk
import math
import numpy as np
import warnings
from sklearn.model_selection import cross_validate


warnings.filterwarnings("ignore")
from sklearn.pipeline import make_pipeline, Pipeline
from importlib import reload

In [2]:
import dodflib.core.loader as LD
from dodflib.core.utils import pad_dataframe

import dodflib.core.preprocess as P
import dodflib.core.feature_extraction as FT
import dodflib.core.utils as UT
import dodflib.core.transformers as T

from importlib import reload as rl

In [3]:
anc = 'Ato_Nomeacao_Comissionado'
aec = 'Ato_Exoneracao_Comissionado'

In [4]:
reload(LD)
rl(UT)
p = ['xmls/10_193.17.9.2013_v1.xml']
aec = 'Ato_Exoneracao_Comissionado' 
ldr = LD.LoaderXML(p)

In [5]:
ldr.all_columns

{'Ato_Abono_Permanencia',
 'Ato_Cessao',
 'Ato_Exoneracao_Comissionado',
 'Ato_Exoneracao_Efetivo',
 'Ato_Nomeacao_Comissionado',
 'Ato_Nomeacao_Efetivo',
 'Ato_Retificacao_Comissionado',
 'Ato_Retificacao_Efetivo',
 'Ato_Reversao',
 'Ato_Substituicao',
 'Ato_Tornado_Sem_Efeito_Apo',
 'Ato_Tornado_Sem_Efeito_Exo_Nom',
 'a_pedido_ou_nao',
 'candidato',
 'candidato_PNE',
 'cargo',
 'cargo_comissionado',
 'cargo_efetivo',
 'cargo_objeto_substituicao',
 'cargo_orgao_cessionario',
 'cargo_substituto',
 'carreira',
 'classe',
 'data_documento',
 'data_dodf',
 'data_dodf_edital_normativo',
 'data_dodf_resultado_final',
 'data_edital_normativo',
 'data_edital_resultado_final',
 'data_final',
 'data_inicial',
 'edital_normativo',
 'edital_resultado_final',
 'especialidade',
 'fundamento_legal',
 'hierarquia_lotacao',
 'informacao_corrigida',
 'informacao_errada',
 'lotacao',
 'matricula',
 'matricula_SIAPE',
 'matricula_substituido',
 'matricula_substituto',
 'matriucla_SIAPE',
 'motivo',
 'nom

In [6]:
reload(P)
reload(UT)


params = dict(
    algorithm = 'l2sgd', 
    c2=1,
    max_iterations=10, 
    all_possible_transitions=True,
)

model = sklearn_crfsuite.CRF(
    **params,
    verbose=False
)


to_drop=r'(annotator|offset)'
pipe_crf = Pipeline([
    ('drop_pattern', T.DropPattern(to_drop)),
    ('selector', T.SelectColumn(aec)),
    ('tokenizer', T.Tokenizer()),
    ('crformer', T.CRFFeaturizer()),
    ('model', sklearn_crfsuite.CRF(**params, verbose=False)),
])

pipe_labelizer = Pipeline([
    ('drop_pattern', T.DropPattern(to_drop)),
    ('keep_only', T.KeepOnlyOneAct(aec)),
    ('iobifier', T.IOBifyer(column=aec)),
])

df = ldr[aec]
train_y=pipe_labelizer.fit_transform(df);
pipe_crf.fit(df, train_y);

In [7]:
import math
training_ratio = 0.7
limit = int(math.ceil(training_ratio * df.shape[0]))


df_train, df_test = df.iloc[:limit], df.iloc[limit:]
y = pipe_labelizer.fit_transform(df_test)

pipe_crf.fit(
    df_train,
    pipe_labelizer.fit_transform(df_train)
);
y_pred=pipe_crf.predict(df_test);

In [8]:
y

0     [O, B-nome, I-nome, I-nome, I-nome, B-cargo_ef...
1     [O, B-nome, I-nome, I-nome, I-nome, B-cargo_ef...
2     [O, B-a_pedido_ou_nao, I-a_pedido_ou_nao, B-no...
3     [O, B-a_pedido_ou_nao, I-a_pedido_ou_nao, B-no...
4     [O, B-a_pedido_ou_nao, I-a_pedido_ou_nao, B-no...
5     [O, B-a_pedido_ou_nao, I-a_pedido_ou_nao, B-no...
6     [O, B-a_pedido_ou_nao, I-a_pedido_ou_nao, B-no...
7     [O, B-a_pedido_ou_nao, I-a_pedido_ou_nao, B-no...
8     [O, B-nome, I-nome, I-nome, I-nome, B-cargo_ef...
9     [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
10    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
11    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
dtype: object